In [1]:
!pip install -U plotly kaleido

In [2]:
#!wget https://huggingface.co/spaces/lmsys/mt-bench/resolve/main/data/mt_bench/model_judgment/gpt-4_single.jsonl

In [3]:
!pip install plotly

In [1]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


CATEGORIES = ["Writing", "Roleplay", "Reasoning", "Math", "Coding", "Extraction", "STEM", "Humanities"]


def get_model_df():
    cnt = 0
    q2result = []
    fin = open("gpt-4_single.jsonl", "r")
    for line in fin:
        obj = json.loads(line)
        obj["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        q2result.append(obj)
    df = pd.DataFrame(q2result)
    return df

def toggle(res_str):
    if res_str == "win":
        return "loss"
    elif res_str == "loss":
        return "win"
    return "tie"

def get_model_df_pair():
    fin = open("gpt-4_pair.jsonl", "r")
    cnt = 0
    q2result = []
    for line in fin:
        obj = json.loads(line)

        result = {}
        result["qid"] = str(obj["question_id"])
        result["turn"] = str(obj["turn"])
        if obj["g1_winner"] == "model_1" and obj["g2_winner"] == "model_1":
            result["result"] = "win"
        elif obj["g1_winner"] == "model_2" and obj["g2_winner"] == "model_2":
            result["result"] = "loss"
        else:
            result["result"] = "tie"
        result["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        result["model"] = obj["model_1"]
        q2result.append(result)

    df = pd.DataFrame(q2result)

    return df

df = get_model_df()
#df_pair = get_model_df_pair()

In [2]:
all_models = df["model"].unique()
print(all_models)
scores_all = []
for model in all_models:
    for cat in CATEGORIES:
        # filter category/model, and score format error (<1% case)
        res = df[(df["category"]==cat) & (df["model"]==model) & (df["score"] >= 0)]
        score = res["score"].mean()

        # # pairwise result
        # res_pair = df_pair[(df_pair["category"]==cat) & (df_pair["model"]==model)]["result"].value_counts()
        # wincnt = res_pair["win"] if "win" in res_pair.index else 0
        # tiecnt = res_pair["tie"] if "tie" in res_pair.index else 0
        # winrate = wincnt/res_pair.sum()
        # winrate_adjusted = (wincnt + tiecnt)/res_pair.sum()
        # # print(winrate_adjusted)

        # scores_all.append({"model": model, "category": cat, "score": score, "winrate": winrate, "wtrate": winrate_adjusted})
        scores_all.append({"model": model, "category": cat, "score": score})

['mistral-7b-sft-beta' 'gemma_dpo' 'gemma_rdpo_eta0.005_no_decay'
 'zephyr-7b-gemma-sft-v0.1' 'zephyr-7b-gemma-v0.1'
 'beta_ultra_rdpo_full_eta0.005_beta0.01_no_decay_new'
 'beta_ultra_dpo_full_beta0.01_new']


In [5]:
target_models = ["gemma_dpo", "gemma_rdpo_eta0.005_no_decay", #"beta_ultra_rdpo_full_eta0.005_beta0.01_stepwise_end0.5_new",
                 #"zephyr-7b-gemma-v0.1", 
                 "zephyr-7b-gemma-sft-v0.1"]

scores_target = [scores_all[i] for i in range(len(scores_all)) if scores_all[i]["model"] in target_models]

# sort by target_models
scores_target = sorted(scores_target, key=lambda x: target_models.index(x["model"]), reverse=True)

df_score = pd.DataFrame(scores_target)
df_score = df_score[df_score["model"].isin(target_models)]

rename_map = {"zephyr-7b-gemma-v0.1": "zephyr-7b-gemma-v0.1",
              "gemma_dpo": "DPO (gemma)",
              "gemma_rdpo_eta0.005_no_decay": "RPO (gemma)",
              #"beta_ultra_rdpo_full_eta0.005_beta0.01_no_decay_new": "RPO (eta=0.005)",
              "zephyr-7b-gemma-sft-v0.1": "Base (gemma)"}

for k, v in rename_map.items():
    df_score.replace(k, v, inplace=True)

fig = px.line_polar(df_score, r = 'score', theta = 'category', line_close = True, category_orders = {"category": CATEGORIES},
                    color = 'model', markers=True, color_discrete_sequence=px.colors.qualitative.Pastel)

fig.show()

: 

In [5]:
target_models = ["beta_ultra_dpo_full_beta0.01_new", "beta_ultra_rdpo_full_eta0.005_beta0.01_no_decay_new", #"beta_ultra_rdpo_full_eta0.005_beta0.01_stepwise_end0.5_new",
                 #"zephyr-7b-gemma-v0.1", 
                 "mistral-7b-sft-beta"]

scores_target = [scores_all[i] for i in range(len(scores_all)) if scores_all[i]["model"] in target_models]

# sort by target_models
scores_target = sorted(scores_target, key=lambda x: target_models.index(x["model"]), reverse=True)

df_score = pd.DataFrame(scores_target)
df_score = df_score[df_score["model"].isin(target_models)]

rename_map = {"zephyr-7b-beta": "zephyr-7b-beta",
              "beta_ultra_dpo_full_beta0.01_new": "DPO (beta)",
              "beta_ultra_rdpo_full_eta0.005_beta0.01_no_decay_new": "RPO (beta)",
              #"beta_ultra_rdpo_full_eta0.005_beta0.01_no_decay_new": "RPO (eta=0.005)",
              "mistral-7b-sft-beta": "Base (beta)"}

for k, v in rename_map.items():
    df_score.replace(k, v, inplace=True)

fig = px.line_polar(df_score, r = 'score', theta = 'category', line_close = True, category_orders = {"category": CATEGORIES},
                    color = 'model', markers=True, color_discrete_sequence=px.colors.qualitative.Pastel)

fig.show()